# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

In [38]:
import pandas as pd
import numpy as np
from scipy import stats

In [39]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [40]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [41]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### Hypothesis Test Comparing Population Proportions
I will use a hypothesis test that relies on the CLT to determine if I can reject the null hypothesis.   
##### Null Hypothesis
There is no difference between call backs for black-sounding and white-sounding names.
##### Alternate Hypothesis
There is a difference between call backs for black-sounding and white-sounding names. Black-sounding names are less likely to receive a call back. 

In [42]:
#white and black data sets
white = data[data['race']=='w']
black = data[data['race']=='b']
#number of callbacks for each race
w_cb = sum(data[data.race=='w'].call)
b_cb = sum(data[data.race=='b'].call)
#number of resumes for each race
w_n = len(white)
b_n = len(black)
#mean and p value
w_m_p = round(w_cb/w_n, 3)
b_m_p = round(b_cb/b_n, 3)
#variance to use to calculate difference of std
w_var = round(w_m_p*(1-w_m_p)/w_n, 6)
b_var = round(b_m_p*(1-b_m_p)/b_n, 6)


In [43]:
print('Statisitcs for white sounding names:')
print('Number of call backs: ' + str(w_cb))
print('Total number of resumes: ' + str(w_n))
print('P-value of call backs: ' + str(round(b_cb/b_n, 3)))
print('Mean: ' + str(w_m_p))
print('Variance: ' + str(w_var))
print('')
print('Statisitcs for black sounding names:')
print('Number of call backs: ' + str(b_cb))
print('Total number of resumes: ' + str(b_n))
print('P-value of call backs: ' + str(b_m_p))
print('Mean: ' + str(b_m_p))
print('Variance: ' + str(b_var))


Statisitcs for white sounding names:
Number of call backs: 235.0
Total number of resumes: 2435
P-value of call backs: 0.064
Mean: 0.097
Variance: 3.6e-05

Statisitcs for black sounding names:
Number of call backs: 157.0
Total number of resumes: 2435
P-value of call backs: 0.064
Mean: 0.064
Variance: 2.5e-05


### Test Null Hypothesis
If p < 5% reject the null hypothesis

In [44]:
#differences in white-sounding names and black-sounding names means
diff_mean = w_m_p -  b_m_p
#adding n values for total sample size
joint_n = w_n + b_n
#mean for the null hypothesis - assuming there is no difference between the means
null_mean = 0
#calculating joint probability
joint_p = round((w_cb + b_cb)/(w_n + b_n),2)
#using variable as short cut for 1-p in variance calculation
p_1 = 1 - joint_p
#variance using (p*(1-p) + p*(1-p)/(n1+n2))
joint_var = 2 * joint_p * p_1 / joint_n
joint_std = round(np.sqrt(joint_var), 4)
#finding the z_score for the difference of sample means
z_sample = round(diff_mean/joint_std ,4)

In [45]:
print('Difference in means: ' + str(diff_mean))
print('Null hypothesis mean: ' + str(null_mean))
print('Standard deviation: ' + str(joint_std))
print('Z-score for difference of sample means: ' + str(z_sample))
print('P value for difference of means: .0000003%')


Difference in means: 0.033
Null hypothesis mean: 0
Standard deviation: 0.0055
Z-score for difference of sample means: 6.0
P value for difference of means: .0000003%


### Reject Null Hypothesis
The z-score for the difference of our sample means is 6.0, which is greater than 1.65, the z-score used to calculate a 95% confidence interval. Using a z-score calculator, I discovered the p-value of our sample is .000000003, therefore allowing us to confidently reject the null hypothesis. 

Since we can reject the null hypothesis, we assume that there is a difference between how many call backs black sounding names and white sounding names will recieve. 

In [46]:
#creating 1-p variable to make it easier to write variance 
pb_1 = 1 - b_m_p
pw_1 = 1 - w_m_p
#variance using (p1*(1-p1)/n1 + p2*(1-p2)/n2)
var = (b_m_p*pb_1/b_n) + (w_m_p*pw_1/w_n)
#standard deviation - taking square root of variance 
std = round(np.sqrt(var), 3)
#margin of error
marg_err = 1.65 * std
#confidence interval
conf_int1 = diff_mean - marg_err
conf_int2 = diff_mean + marg_err

In [47]:
print('Difference in means: ' + str(diff_mean))
print('Standard deviation: ' + str(std))
print('Margin of Error: + or - ' + str(marg_err))
print('95% Confidence Interval: ' + str(conf_int1) + '-' + str(conf_int2))

Difference in means: 0.033
Standard deviation: 0.008
Margin of Error: + or - 0.0132
95% Confidence Interval: 0.0198-0.0462


### Summary
I am confident that there is a 95% chance that white-sounding names will recieve between 2% and 4.6% more callbacks than black-sounding names. The margin of error is 1.3% 
The analysis shows that race is a factor in the amount of call backs an individual recieves. 